In [1]:

import os

import torch

from torch_geometric.nn import Node2Vec

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [2]:
def adjacency_to_edge_index(file):
    edge_index = [[], []]
    for indx, line in enumerate(file):
        for edge in line.split(","):
            edge_index[0].append(indx)
            edge_index[1].append(int(edge))
    
    return edge_index

In [3]:
dataset_dir = os.path.join(
    os.getenv("CVF_PROJECT_DIR", ""), "cvf-analysis", "v2", "datasets", "coloring"
)
edge_index_file = "tiny_graph_test_pt_adj_list.txt"

f = open(os.path.join(dataset_dir, edge_index_file), "r")
edge_index = torch.tensor(adjacency_to_edge_index(f)).to(device)


In [4]:
edge_index

tensor([[ 0,  0,  0,  1,  1,  2,  2,  3,  4,  5,  5,  6,  6,  7,  7,  7,  8,  9,
         10, 11],
        [ 4,  2,  1,  9,  3, 10,  3,  3,  4,  9,  4, 10,  4,  3,  5,  6,  8,  9,
         10, 11]], device='cuda:0')

In [5]:
model = Node2Vec(
    edge_index,
    embedding_dim=2,
    walks_per_node=5,
    walk_length=5,
    context_size=3,
    p=1.0,
    q=1.0,
    num_negative_samples=1,
).to(device)


In [6]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [7]:
# @torch.no_grad()
# def test():
#     model.eval()
#     z = model()
#     acc = model.test(
#         train_z=z[data.train_mask],
#         train_y=data.y[data.train_mask],
#         test_z=z[data.test_mask],
#         test_y=data.y[data.test_mask],
#         max_iter=150,
#     )
#     return acc

In [8]:
for epoch in range(1, 101):
    loss = train()
    # acc = test()
    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


Epoch: 001, Loss: 1.1533
Epoch: 002, Loss: 1.1657
Epoch: 003, Loss: 1.1808
Epoch: 004, Loss: 1.1843
Epoch: 005, Loss: 1.1270
Epoch: 006, Loss: 1.1179
Epoch: 007, Loss: 1.2118
Epoch: 008, Loss: 1.1792
Epoch: 009, Loss: 1.1430
Epoch: 010, Loss: 1.1334
Epoch: 011, Loss: 1.1535
Epoch: 012, Loss: 1.1341
Epoch: 013, Loss: 1.1643
Epoch: 014, Loss: 1.0889
Epoch: 015, Loss: 1.1588
Epoch: 016, Loss: 1.1630
Epoch: 017, Loss: 1.1596
Epoch: 018, Loss: 1.1011
Epoch: 019, Loss: 1.1040
Epoch: 020, Loss: 1.1716
Epoch: 021, Loss: 1.1769
Epoch: 022, Loss: 1.1337
Epoch: 023, Loss: 1.1526
Epoch: 024, Loss: 1.1343
Epoch: 025, Loss: 1.1168
Epoch: 026, Loss: 1.1228
Epoch: 027, Loss: 1.0801
Epoch: 028, Loss: 1.0478
Epoch: 029, Loss: 1.1279
Epoch: 030, Loss: 1.0923
Epoch: 031, Loss: 1.1212
Epoch: 032, Loss: 1.1185
Epoch: 033, Loss: 1.1181
Epoch: 034, Loss: 1.0490
Epoch: 035, Loss: 1.0571
Epoch: 036, Loss: 1.0753
Epoch: 037, Loss: 1.0531
Epoch: 038, Loss: 1.0927
Epoch: 039, Loss: 1.0301
Epoch: 040, Loss: 1.1085


In [9]:
model()

Parameter containing:
tensor([[-0.0723, -0.5599],
        [-0.7039, -0.6210],
        [ 0.2282,  0.1689],
        [ 0.9819, -1.3159],
        [-1.4171, -0.8673],
        [-0.0280,  0.0837],
        [-0.4404,  0.2050],
        [ 0.9470,  0.2032],
        [ 1.2039,  0.8226],
        [ 0.5642, -1.4475],
        [-0.5823,  1.5101],
        [-0.9096,  0.9855]], device='cuda:0', requires_grad=True)